# Summary

Using the drivers rasters from GEE tlg_habitat_intactness.ipynb and locally to combine them into Habitat Intactness rasters for Majete Wildlife Reserve, Malawi.


# Import AOI and setup

In [1]:
import os
from google.cloud import storage
import rasterio
from rasterio.plot import show
import numpy as np
from scipy.ndimage import generic_filter
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray
import dask_gateway

In [ ]:
cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.scale(4)
print(cluster.dashboard_link)

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/gc_key.json"

In [ ]:
def download_blob(bucket_name, blob_name, local_dir):
    """Downloads a blob from the bucket to a specified directory."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Construct file path in the specified local directory
    local_file_path = os.path.join(local_dir, blob_name)

    # Create directories if they don't exist
    os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

    # Download the file
    blob.download_to_filename(local_file_path)
    return local_file_path


In [ ]:
bucket_name = 'nature-watch-bucket'
blob_name = 'COGS/all/2023/all2023_15.tif'
local_dir = '../data/nature-watch/'

all_image = download_blob(bucket_name, blob_name, local_dir)


In [ ]:
with rasterio.open(all_image) as r:
    if r.is_tiled:
        print(f"Chunk size: {r.block_shapes}")

In [ ]:
raster_ds = rioxarray.open_rasterio(all_image, lock=False, chunks="auto")

In [ ]:
def quart3(window):
    return np.percentile(window, 75)

def quart3_xarray(da, kernel_size=3):
    return xr.apply_ufunc(
        lambda x: generic_filter(x, quart3, size=kernel_size),
        da,
        input_core_dims=[['y', 'x']],
        output_core_dims=[['y', 'x']],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[da.dtype]
    )

In [ ]:
filtered_raster = quart3_xarray(raster_ds, kernel_size=5)
filtered_raster.compute()

In [ ]:
plt.imshow(filtered_raster, cmap='gray')
plt.colorbar()
plt.show()